In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np 
from pathlib import Path
from rtseg.utils.param_io import load_params
import glob
import polars as pl
from rtseg.cells.plotting import fast_generate_fork_plot, get_bulk_init_area, slice_fork_plot_around_init
import matplotlib.pyplot as plt
%matplotlib qt5
from rtseg.utils.get_fork_init import read_all_fork_data_around_init
import rtseg.cells.scoring as sco 
import pysal.lib
from rtseg.cells.scoring import energy_dists
from cycler import cycler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy.io 
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull, Delaunay
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import h5py

In [4]:
param = load_params(Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0125/expt_params.yaml'), ref_type='expt')
param.Save.directory = 'C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0125/'
save_dir = Path(param.Save.directory)

#### Fork plots for all traps

In [5]:
nr_pos = 32
pos_inds = np.vstack((np.arange(0,nr_pos//4), np.arange(nr_pos//4, nr_pos//2), np.arange(nr_pos//2,3*nr_pos//4), np.arange(3*nr_pos//4,nr_pos)))
nr_strains = pos_inds.shape[0]
nr_pos_per_strain = nr_pos//nr_strains
titles = ['Reference', 'ΔDARS1', 'ΔDARS2', 'ΔdatA']

In [6]:
use_pyarrow = False

fork_filenames = glob.glob(str(save_dir / Path('Pos[0-9]*/forks.parquet')))
columns_to_extract = ['area', 'length', 'normalized_internal_x', 'normalization_counts', 'position', 'trap']
data = pl.read_parquet(fork_filenames, use_pyarrow=use_pyarrow, columns=columns_to_extract)

areas = data['area'].to_numpy()
lengths = data['length'].to_numpy()
longs = data['normalized_internal_x'].to_numpy()
counts = data['normalization_counts'].to_numpy()
position = data['position'].to_numpy()
all_traps = data['trap'].to_numpy()
bin_scale = param.Forkplots.bin_scale
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size
min_length = param.Forkplots.min_length
arb_div_area = param.Forkplots.arb_div_area
init_area_cv = param.Forkplots.init_area_cv
uni_pos = np.unique(position)
all_traps = data['trap'].to_numpy()
nr_traps = param.BarcodeAndChannels.num_blocks_per_image*param.BarcodeAndChannels.num_traps_per_block
trap_nrs = np.arange(nr_traps)

In [7]:
plt.close('all')

init_area_ref = None
heatmap_init_ref = None
moran_weight = None
e_dists = None
fork_plot_save_dir = Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0128/ForkPlots')
for i in range(0, nr_strains):
    strain_pos_inds = np.where(np.isin(position, pos_inds[i,:]+1))[0]
    strain_areas = areas[strain_pos_inds]
    strain_lengths = lengths[strain_pos_inds]
    strain_longs = longs[strain_pos_inds]
    strain_counts = counts[strain_pos_inds]
    strain_traps = all_traps[strain_pos_inds]
    strain_pos = position[strain_pos_inds]

    if i == 0:    
        heatmap, mean_cell_lengths, abins, lbins, (x, y) = fast_generate_fork_plot(strain_areas, strain_lengths, strain_longs, 
                                                                                   strain_counts, bin_scale=bin_scale, pixel_size=pixel_size, 
                                                                                   heatmap_threshold=heatmap_threshold)
    else:
        heatmap, mean_cell_lengths, _, _, _ = fast_generate_fork_plot(strain_areas, strain_lengths, strain_longs, strain_counts, 
                                                                      bin_scale=bin_scale, pixel_size=pixel_size,
                                                                      heatmap_threshold=heatmap_threshold, abins=abins, lbins=lbins)

    init_area = get_bulk_init_area(strain_areas, strain_counts, strain_longs, strain_lengths, min_length=min_length, 
                                   pixel_size=pixel_size, arb_div_area=arb_div_area)
    if i == 0:
        init_area_ref = init_area
        area_bins_init, length_bins_init, heatmap_init, mean_cell_lengths_init, abins_inds_init, lbins_inds_init = slice_fork_plot_around_init(abins, lbins, heatmap, mean_cell_lengths, init_area, init_area_cv)
        heatmap_init_ref = heatmap_init
        moran_weights = pysal.lib.weights.lat2W(heatmap_init.shape[0], heatmap_init.shape[-1], rook=True)
        e_dists = energy_dists(length_bins_init, area_bins_init)
    else:
        _, _, heatmap_init, mean_cell_lengths_init, _, _ = slice_fork_plot_around_init(abins, lbins, heatmap, mean_cell_lengths, 
                                                                                       init_area_ref, init_area_cv)
    
    if i == 0:
        plot_heatmap = plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper', cmap='jet')
        color_lims = (plot_heatmap.norm.vmin, plot_heatmap.norm.vmax)
    else:
        plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper', cmap='jet', 
                    vmin=color_lims[0], vmax=color_lims[-1])
        
    plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
    plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
    plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
    plt.xlim(-3, 3) 
    plt.ylim(3,y[0])
    plt.xlabel('Cell long axis (µm)')
    plt.ylabel('Cell size (µm$^2$)')
    plt.title(titles[i])

    fork_plot_filename_pdf = f'ForkPlot_{titles[i]}.pdf'
    fork_plot_filename_svg = f'ForkPlot_{titles[i]}.svg' 
    fork_plot_pdf_path = fork_plot_save_dir / fork_plot_filename_pdf 
    fork_plot_svg_path = fork_plot_save_dir / fork_plot_filename_svg
    plt.savefig(fork_plot_pdf_path, transparent=True)
    plt.savefig(fork_plot_svg_path, transparent=True)
    
    plt.matshow(heatmap_init, aspect='auto', interpolation='none', 
            extent=[length_bins_init[0], length_bins_init[-1], area_bins_init[-1], area_bins_init[0]], origin='upper', cmap='jet', 
               vmin=color_lims[0], vmax=color_lims[-1])
    plt.plot(-0.5 * mean_cell_lengths_init, y[abins_inds_init[0:-1]], 'w', linewidth=2)
    plt.plot(0.5 * mean_cell_lengths_init, y[abins_inds_init[0:-1]], 'w', linewidth=2)
    plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
    plt.xlabel('Cell long axis (µm)')
    plt.ylabel('Cell size (µm$^2$)')
    plt.title(titles[i])

    fork_plot_init_filename_pdf = f'ForkPlotInit_{titles[i]}.pdf'
    fork_plot_init_filename_svg = f'ForkPlotInit_{titles[i]}.svg' 
    fork_plot_init_pdf_path = fork_plot_save_dir / fork_plot_init_filename_pdf 
    fork_plot_init_svg_path = fork_plot_save_dir / fork_plot_init_filename_svg 
    plt.savefig(Path(fork_plot_save_dir / fork_plot_init_filename_pdf), transparent=True)
    plt.savefig(Path(fork_plot_save_dir / fork_plot_init_filename_svg), transparent=True)
    print(f"{titles[i]}, number of dots: {len(strain_areas)}")

Reference, number of dots: 1869127


C:\Users\oscbr226\Desktop\Initiation_Lib_Tweezer\Code\realtimeAnalysis\rtseg\rtseg\cells\plotting.py:202: RuntimeWarning: invalid value encountered in divide
  heatmap = heatmap/normFactor[:, np.newaxis]


ΔDARS1, number of dots: 1654842
ΔDARS2, number of dots: 1804144


C:\Users\oscbr226\Desktop\Initiation_Lib_Tweezer\Code\realtimeAnalysis\rtseg\rtseg\cells\plotting.py:202: RuntimeWarning: invalid value encountered in divide
  heatmap = heatmap/normFactor[:, np.newaxis]


ΔdatA, number of dots: 2133063


In [8]:
plt.close('all')
amin_init = np.min(area_bins_init)
amax_init = np.max(area_bins_init)
lmin_init = np.min(length_bins_init)
lmax_init = np.max(length_bins_init)

heatmap_init_dims = heatmap_init_ref.shape
heatmap_init_pos_traps = np.zeros(shape=(heatmap_init_dims[0], heatmap_init_dims[1], nr_pos, nr_traps))
flat_heatmap_init_pos_traps = np.zeros(shape=(heatmap_init_dims[0]*heatmap_init_dims[1], nr_pos, nr_traps))
all_traps_nr_dots = np.zeros(shape=(nr_pos, nr_traps))
for i, j, in (np.ndindex(nr_pos, nr_traps)):
    pos = uni_pos[i]
    trap = trap_nrs[j]

    filename = save_dir / Path('Pos' + str(pos)) / Path('forks.parquet/trap=' + str(trap))
    if not filename.exists():
        continue

    trap_inds = np.where((position == pos) & (all_traps == trap))
    areas_trap = areas[trap_inds]
    longs_trap = longs[trap_inds]
    lengths_trap = lengths[trap_inds]
    counts_trap = counts[trap_inds] 
    
    heatmap_trap, mean_cell_lengths_trap, _, _, _ = fast_generate_fork_plot(areas_trap, lengths_trap, longs_trap, 
                                                               counts_trap, abins=abins, lbins=lbins,
                                                               bin_scale=bin_scale, pixel_size=pixel_size,
                                                               heatmap_threshold=heatmap_threshold)
            
    heatmap_trap_init = heatmap_trap[np.ix_(abins_inds_init, lbins_inds_init)]
    mean_cell_lengths_trap_init = mean_cell_lengths_trap[abins_inds_init[0:-1]]
    flat_heatmap_trap_init = heatmap_trap_init.flatten()
    heatmap_init_pos_traps[:,:,i,j] = heatmap_trap_init
    flat_heatmap_init_pos_traps[:,i,j] = flat_heatmap_trap_init


    
    #plt.matshow(heatmap_trap_init, aspect='auto', interpolation='none', 
    #        extent=[length_bins_init[0], length_bins_init[-1], area_bins_init[-1], area_bins_init[0]], origin='upper', cmap='jet', 
    #           vmin=color_lims[0], vmax=color_lims[-1])
    #plt.plot(-0.5 * mean_cell_lengths_trap_init, y[abins_inds_init[0:-1]], 'w', linewidth=2)
    #plt.plot(0.5 * mean_cell_lengths_trap_init, y[abins_inds_init[0:-1]], 'w', linewidth=2)
    #plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
    #plt.xlabel('Cell long axis (µm)')
    #plt.ylabel('Cell size (µm$^2$)')
    #plt.title(titles[i])

    #if j == 2:
    #    break
    #Getting the number of dots around initiation

    ar_pix_trap = areas_trap * pixel_size**2
    le_pix_trap = lengths_trap * pixel_size
    lo_cent_trap = longs_trap - 0.5
    len_long_rescaled_trap = le_pix_trap * lo_cent_trap
    inds_around_init_trap = np.where((ar_pix_trap >= amin_init) & (ar_pix_trap <= amax_init) & 
                                        (len_long_rescaled_trap >= lmin_init) & 
                                        (len_long_rescaled_trap <= lmax_init))[0]
    lo_init_trap = lo_cent_trap[inds_around_init_trap]
    nr_dots_init_trap = lo_init_trap.size
    all_traps_nr_dots[i,j] = nr_dots_init_trap

C:\Users\oscbr226\Desktop\Initiation_Lib_Tweezer\Code\realtimeAnalysis\rtseg\rtseg\cells\plotting.py:202: RuntimeWarning: invalid value encountered in divide
  heatmap = heatmap/normFactor[:, np.newaxis]


#### Doing scoring on a per trap basis

In [9]:
corrcoeffs = np.full((nr_pos, nr_traps), np.nan)
ssims = np.full((nr_pos, nr_traps), np.nan)
morans = np.full((nr_pos, nr_traps, 3), np.nan)
ks_results = np.full((nr_pos, nr_traps, 2), np.nan)
sobolevs = np.full((nr_pos, nr_traps), np.nan)
energies = np.full((nr_pos, nr_traps), np.nan)

flat_heatmap_init_ref = heatmap_init_ref.flatten()

In [10]:
for (i, j) in (np.ndindex(nr_pos, nr_traps)):
        flat_trap_heatmap = flat_heatmap_init_pos_traps[:,i,j]
        trap_heatmap = heatmap_init_pos_traps[:,:,i,j]
        
        if not np.any(flat_trap_heatmap):
            #Skips trap if there are no data points within the scoring window
            continue
        
        corrcoeffs[i,j] = sco.score_correlation_coefficient(flat_heatmap_init_ref, flat_trap_heatmap)

        ssims[i,j] = sco.score_ssim(flat_heatmap_init_ref, flat_trap_heatmap)

        morans[i,j,0], morans[i,j,1], morans[i,j,2] = sco.score_cross_moran(flat_heatmap_init_ref, flat_trap_heatmap, moran_weights)

        ks_results[i,j,0], ks_results[i,j,1] = sco.score_kolmogorov_smirnov(flat_heatmap_init_ref, flat_trap_heatmap)

        sobolevs[i,j] = sco.score_sobolev_norm(heatmap_init_ref, trap_heatmap)

        energies[i,j] = sco.score_energy_test(flat_heatmap_init_ref, flat_trap_heatmap, e_dists)

#### Plot scores

In [9]:
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams['lines.markersize'] = 4
plt.rcParams['axes.prop_cycle'] = cycler(color=CB_color_cycle)

In [13]:
plt.close('all')

fig_corr, ax_corr = plt.subplots()
fig_ssim, ax_ssim = plt.subplots()
fig_moran, ax_moran = plt.subplots()
fig_ks, ax_ks = plt.subplots()
fig_sobolev, ax_sobolev = plt.subplots()
fig_energy, ax_energy = plt.subplots()

fig_hist_corr, ax_hist_corr = plt.subplots()
fig_hist_ssim, ax_hist_ssim = plt.subplots()
fig_hist_moran, ax_hist_moran = plt.subplots()
fig_hist_ks, ax_hist_ks = plt.subplots()
fig_hist_sobolev, ax_hist_sobolev = plt.subplots()
fig_hist_energy, ax_hist_energy = plt.subplots()

fig_dots_corr, ax_dots_corr = plt.subplots()
fig_dots_ssim, ax_dots_ssim = plt.subplots()
fig_dots_moran, ax_dots_moran = plt.subplots()
fig_dots_ks, ax_dots_ks = plt.subplots()
fig_dots_sobolev, ax_dots_sobolev = plt.subplots()
fig_dots_energy, ax_dots_energy = plt.subplots()

tot_strain_nr_traps = nr_traps*nr_pos_per_strain
plot_range = np.arange(1, tot_strain_nr_traps+1, 1)
nr_bins = 30
for i in range(0, nr_strains):
    strain_correlations = corrcoeffs[pos_inds[i],:].flatten()
    strain_ssims = ssims[pos_inds[i],:].flatten()
    strain_moran = morans[pos_inds[i],:,0].flatten()
    strain_ks = ks_results[pos_inds[i],:,0].flatten()
    strain_sobolev = sobolevs[pos_inds[i],:].flatten()
    strain_energies = energies[pos_inds[i],:].flatten()
    strain_nr_dots = all_traps_nr_dots[pos_inds[i],:].flatten()

    ax_corr.plot(plot_range, strain_correlations, 'o', label=titles[i])
    ax_ssim.plot(plot_range, strain_ssims, 'o', label=titles[i])
    ax_moran.plot(plot_range, strain_moran, 'o', label=titles[i])
    ax_ks.plot(plot_range, strain_ks, 'o', label=titles[i])
    ax_sobolev.plot(plot_range, strain_sobolev, 'o', label=titles[i])
    ax_energy.plot(plot_range, strain_energies, 'o', label=titles[i])

    corr_hist, corr_bins = np.histogram(strain_correlations[~np.isnan(strain_correlations)], nr_bins)
    ssim_hist, ssim_bins = np.histogram(strain_ssims[~np.isnan(strain_ssims)], nr_bins)
    moran_hist, moran_bins = np.histogram(strain_moran[~np.isnan(strain_moran)], nr_bins)
    ks_hist, ks_bins = np.histogram(strain_ks[~np.isnan(strain_ks)], nr_bins)
    sobolev_hist, sobolev_bins = np.histogram(strain_sobolev[~np.isnan(strain_sobolev)], nr_bins)
    energy_hist, energy_bins = np.histogram(strain_energies[~np.isnan(strain_energies)], nr_bins)

    corr_bin_cent = 0.5 * (corr_bins[1:] + corr_bins[:-1])
    ssim_bin_cent = 0.5 * (ssim_bins[1:] + ssim_bins[:-1]) 
    moran_bin_cent = 0.5 * (moran_bins[1:] + moran_bins[:-1]) 
    ks_bin_cent = 0.5 * (ks_bins[1:] + ks_bins[:-1]) 
    sobolev_bin_cent = 0.5 * (sobolev_bins[1:] + sobolev_bins[:-1]) 
    energy_bin_cent = 0.5 * (energy_bins[1:] + energy_bins[:-1]) 
    
    ax_hist_corr.plot(corr_bin_cent, corr_hist, label=titles[i])
    ax_hist_ssim.plot(ssim_bin_cent, ssim_hist, label=titles[i])
    ax_hist_moran.plot(moran_bin_cent, moran_hist, label=titles[i])
    ax_hist_ks.plot(ks_bin_cent, ks_hist, label=titles[i])
    ax_hist_sobolev.plot(sobolev_bin_cent, sobolev_hist, label=titles[i])
    ax_hist_energy.plot(energy_bin_cent, energy_hist, label=titles[i])

    ax_dots_corr.plot(strain_nr_dots, strain_correlations, 'o', label=titles[i])
    ax_dots_ssim.plot(strain_nr_dots, strain_ssims, 'o', label=titles[i])
    ax_dots_moran.plot(strain_nr_dots, strain_moran, 'o', label=titles[i])
    ax_dots_ks.plot(strain_nr_dots, strain_ks, 'o', label=titles[i])
    ax_dots_sobolev.plot(strain_nr_dots, strain_sobolev, 'o', label=titles[i])
    ax_dots_energy.plot(strain_nr_dots, strain_energies, 'o', label=titles[i])

    print(f"{titles[i]}, number of dots around initiation: {int(np.sum(strain_nr_dots))}")

    #ax_hist_corr.hist(strain_correlations, bins=nr_bins, edgecolor='black', alpha=0.5, label=titles[i])
    #ax_hist_ssim.hist(strain_ssims, bins=nr_bins, edgecolor='black', alpha=0.5, label=titles[i])
    #ax_hist_moran.hist(strain_moran, bins=nr_bins, edgecolor='black', alpha=0.5, label=titles[i])
    #ax_hist_ks.hist(strain_ks, bins=nr_bins, edgecolor='black', alpha=0.5, label=titles[i])
    #ax_hist_sobolev.hist(strain_sobolev, bins=nr_bins, edgecolor='black', alpha=0.5, label=titles[i])
    #ax_hist_energy.hist(strain_energies, bins=nr_bins, edgecolor='black', alpha=0.5, label=titles[i])


Reference, number of dots around initiation: 260118
ΔDARS1, number of dots around initiation: 257094
ΔDARS2, number of dots around initiation: 187766
ΔdatA, number of dots around initiation: 343874


In [16]:
ax_corr.set_title('Strain correlation coefficients')
ax_corr.set_xlabel('Channel')
ax_corr.set_ylabel('Correlation coefficient')
fig_corr.subplots_adjust(right=0.8)
ax_corr.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_ssim.set_title('Strain SSIM')
ax_ssim.set_xlabel('Channel')
ax_ssim.set_ylabel('SSIM')
fig_ssim.subplots_adjust(right=0.8)
ax_ssim.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_moran.set_title('Strain Moran')
ax_moran.set_xlabel('Channel')
ax_moran.set_ylabel('Moran')
fig_moran.subplots_adjust(right=0.8)
ax_moran.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_ks.set_title('Strain KS results')
ax_ks.set_xlabel('Channel')
ax_ks.set_ylabel('KS result')
fig_ks.subplots_adjust(right=0.8)
ax_ks.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_sobolev.set_title('Strain Sobolev norm')
ax_sobolev.set_xlabel('Channel')
ax_sobolev.set_ylabel('Sobolev norm')
fig_sobolev.subplots_adjust(right=0.8)
ax_sobolev.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_energy.set_title('Strain energy test')
ax_energy.set_xlabel('Channel')
ax_energy.set_ylabel('Energy test')
fig_energy.subplots_adjust(right=0.8)
ax_energy.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

In [17]:
ax_hist_corr.set_title('Strain correlation coefficients')
ax_hist_corr.set_ylabel('Count')
ax_hist_corr.set_xlabel('Correlation coefficient')
fig_hist_corr.subplots_adjust(right=0.8)
ax_hist_corr.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_hist_ssim.set_title('Strain SSIM')
ax_hist_ssim.set_ylabel('Count')
ax_hist_ssim.set_xlabel('SSIM')
fig_hist_ssim.subplots_adjust(right=0.8)
ax_hist_ssim.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_hist_moran.set_title('Strain Moran')
ax_hist_moran.set_ylabel('Count')
ax_hist_moran.set_xlabel('Moran')
fig_hist_moran.subplots_adjust(right=0.8)
ax_hist_moran.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_hist_ks.set_title('Strain KS results')
ax_hist_ks.set_ylabel('Count')
ax_hist_ks.set_xlabel('KS result')
fig_hist_ks.subplots_adjust(right=0.8)
ax_hist_ks.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_hist_sobolev.set_title('Strain Sobolev norm')
ax_hist_sobolev.set_ylabel('Count')
ax_hist_sobolev.set_xlabel('Sobolev norm')
fig_hist_sobolev.subplots_adjust(right=0.8)
ax_hist_sobolev.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_hist_energy.set_title('Strain energy test')
ax_hist_energy.set_ylabel('Count')
ax_hist_energy.set_xlabel('Energy test')
fig_hist_energy.subplots_adjust(right=0.8)
ax_hist_energy.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

In [18]:
ax_dots_corr.set_title('Strain correlation coefficients')
ax_dots_corr.set_xlabel('Numberr of dots')
ax_dots_corr.set_ylabel('Correlation coefficient')
fig_dots_corr.subplots_adjust(right=0.8)
ax_dots_corr.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_dots_ssim.set_title('Strain SSIM')
ax_dots_ssim.set_xlabel('Number of dots')
ax_dots_ssim.set_ylabel('SSIM')
fig_dots_ssim.subplots_adjust(right=0.8)
ax_dots_ssim.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_dots_moran.set_title('Strain Moran')
ax_dots_moran.set_xlabel('Number of dots')
ax_dots_moran.set_ylabel('Moran')
fig_dots_moran.subplots_adjust(right=0.8)
ax_dots_moran.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_dots_ks.set_title('Strain KS results')
ax_dots_ks.set_xlabel('Number of dots')
ax_dots_ks.set_ylabel('KS result')
fig_dots_ks.subplots_adjust(right=0.8)
ax_dots_ks.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_dots_sobolev.set_title('Strain Sobolev norm')
ax_dots_sobolev.set_xlabel('Number of dots')
ax_dots_sobolev.set_ylabel('Sobolev norm')
fig_dots_sobolev.subplots_adjust(right=0.8)
ax_dots_sobolev.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

ax_dots_energy.set_title('Strain energy test')
ax_dots_energy.set_xlabel('Number of dots')
ax_dots_energy.set_ylabel('Energy test')
fig_dots_energy.subplots_adjust(right=0.8)
ax_dots_energy.legend(loc='upper left', bbox_to_anchor=(1, 0.65))

#### Save the figures

In [19]:
scoring_plot_dir = Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0128/Scoring')

fig_corr.savefig(scoring_plot_dir / 'CorrelationOverChannels.pdf', transparent='True')
fig_corr.savefig(scoring_plot_dir / 'CorrelationOverChannels.svg', transparent='True')

fig_ssim.savefig(scoring_plot_dir / 'SSIMOverChannels.pdf', transparent='True')
fig_ssim.savefig(scoring_plot_dir / 'SSIMOverChannels.svg', transparent='True')

fig_moran.savefig(scoring_plot_dir / 'MoranOverChannels.pdf', transparent='True')
fig_moran.savefig(scoring_plot_dir / 'MoranOverChannels.svg', transparent='True')

fig_ks.savefig(scoring_plot_dir / 'KSOverChannels.pdf', transparent='True')
fig_ks.savefig(scoring_plot_dir / 'KSOverChannels.svg', transparent='True')

fig_sobolev.savefig(scoring_plot_dir / 'SobolevOverChannels.pdf', transparent='True')
fig_sobolev.savefig(scoring_plot_dir / 'SobolevOverChannels.svg', transparent='True')

fig_energy.savefig(scoring_plot_dir / 'EnergyOverChannels.pdf', transparent='True')
fig_energy.savefig(scoring_plot_dir / 'EnergyOverChannels.svg', transparent='True')

In [20]:
fig_hist_corr.savefig(scoring_plot_dir / 'CorrelationHistogram.pdf', transparent='True')
fig_hist_corr.savefig(scoring_plot_dir / 'CorrelationHistogram.svg', transparent='True')

fig_hist_ssim.savefig(scoring_plot_dir / 'SSIMHistogram.pdf', transparent='True')
fig_hist_ssim.savefig(scoring_plot_dir / 'SSIMHistogram.svg', transparent='True')

fig_hist_moran.savefig(scoring_plot_dir / 'MoranHistogram.pdf', transparent='True')
fig_hist_moran.savefig(scoring_plot_dir / 'MoranHistogram.svg', transparent='True')

fig_hist_ks.savefig(scoring_plot_dir / 'KSHistogram.pdf', transparent='True')
fig_hist_ks.savefig(scoring_plot_dir / 'KSHistogram.svg', transparent='True')

fig_hist_sobolev.savefig(scoring_plot_dir / 'SobolevHistogram.pdf', transparent='True')
fig_hist_sobolev.savefig(scoring_plot_dir / 'SobolevHistogram.svg', transparent='True')

fig_hist_energy.savefig(scoring_plot_dir / 'EnergyHistogram.pdf', transparent='True')
fig_hist_energy.savefig(scoring_plot_dir / 'EnergyHistogram.svg', transparent='True')


In [21]:
fig_dots_corr.savefig(scoring_plot_dir / 'CorrelationOverNrDots.pdf', transparent='True')
fig_dots_corr.savefig(scoring_plot_dir / 'CorrelationOverNrDots.svg', transparent='True')

fig_dots_ssim.savefig(scoring_plot_dir / 'SSIMOverNrDots.pdf', transparent='True')
fig_dots_ssim.savefig(scoring_plot_dir / 'SSIMOverNrDots.svg', transparent='True')

fig_dots_moran.savefig(scoring_plot_dir / 'MoranOverNrDots.pdf', transparent='True')
fig_dots_moran.savefig(scoring_plot_dir / 'MoranOverNrDots.svg', transparent='True')

fig_dots_ks.savefig(scoring_plot_dir / 'KSOverNrDots.pdf', transparent='True')
fig_dots_ks.savefig(scoring_plot_dir / 'KSOverNrDots.svg', transparent='True')

fig_dots_sobolev.savefig(scoring_plot_dir / 'SobolevOverNrDots.pdf', transparent='True')
fig_dots_sobolev.savefig(scoring_plot_dir / 'SobolevOverNrDots.svg', transparent='True')

fig_dots_energy.savefig(scoring_plot_dir / 'EnergyOverNrDots.pdf', transparent='True')
fig_dots_energy.savefig(scoring_plot_dir / 'EnergyOverNrDots.svg', transparent='True')


#### Doing PCA for each strain

In [22]:
plt.close('all')
pca_scaler = StandardScaler()

strains_score_matrix = np.full((nr_pos_per_strain*nr_traps, 6, nr_strains), np.nan)

pca_fig, pca_ax = plt.subplots()
for i in range(0, nr_strains):
    strain_correlations = corrcoeffs[pos_inds[i],:].flatten()
    strain_ssims = ssims[pos_inds[i],:].flatten()
    strain_moran = morans[pos_inds[i],:,0].flatten()
    strain_ks = ks_results[pos_inds[i],:,0].flatten()
    strain_sobolev = sobolevs[pos_inds[i],:].flatten()
    strain_energies = energies[pos_inds[i],:].flatten()

    score_matrix = np.column_stack((strain_correlations, strain_ssims, strain_moran, strain_ks, strain_sobolev, strain_energies))
    strains_score_matrix[:,:,i] = score_matrix 
    non_nan_score_matrix = score_matrix[~np.isnan(score_matrix).any(axis=1)]
    scaled_scores_for_pca = pca_scaler.fit_transform(non_nan_score_matrix)

    pca = PCA(n_components=1)
    reduced_scores = pca.fit_transform(scaled_scores_for_pca)

    #print(non_nan_score_matrix.shape)
    print(pca.explained_variance_ratio_)
    #print(pca)
    pca_ax.plot(reduced_scores, label=titles[i])

pca_fig.subplots_adjust(right=0.8)
pca_ax.set_xlabel('Non-empty trap')
pca_ax.set_ylabel('PCA')
pca_ax.legend(loc='upper left', bbox_to_anchor=(1, 0.65))


[0.72313808]
[0.87529145]
[0.88561867]
[0.62624765]


In [23]:


mat_save_dir = save_dir / 'data_for_pca.mat'
heatmap_save_dir = save_dir / 'heatmaps_init.mat'
scipy.io.savemat(mat_save_dir, {'strain_scores': strains_score_matrix})


#### PCA on fork plots around initiation

In [10]:
strains_trap_heatmaps = np.zeros(shape=(heatmap_init_dims[0], heatmap_init_dims[1], nr_pos_per_strain*nr_traps, nr_strains))
flat_strains_trap_heatmaps = np.zeros(shape=(heatmap_init_dims[0]*heatmap_init_dims[1], nr_pos_per_strain*nr_traps, nr_strains))
for i in range(0, nr_strains):
    dimension_red_heatmap_init = heatmap_init_pos_traps[:,:,pos_inds[i],:].reshape(heatmap_init_dims[0], heatmap_init_dims[1], -1)
    flat_dimension_red_heatmap_init = flat_heatmap_init_pos_traps[:,pos_inds[i],:].reshape(heatmap_init_dims[0]*heatmap_init_dims[1],-1)
    strains_trap_heatmaps[:,:,:,i] = dimension_red_heatmap_init
    flat_strains_trap_heatmaps[:,:,i]= flat_dimension_red_heatmap_init

In [11]:
pooled_strains_flat_heatmap_non_zero = []
nr_non_zero_els = np.zeros(shape=(nr_strains,))
for i in range(0,nr_strains):
    f_str_trap_hm = flat_strains_trap_heatmaps[:,:,i] 
    non_zero_flat_heatmap = f_str_trap_hm[:,np.logical_and(np.all(~np.isnan(f_str_trap_hm), axis=0), np.any(f_str_trap_hm > 0, axis=0))]
    pooled_strains_flat_heatmap_non_zero.append(non_zero_flat_heatmap)
    nr_non_zero_els[i] = non_zero_flat_heatmap.shape[1]


concat_strains_flat_heatmap_non_zero = np.transpose(np.hstack(pooled_strains_flat_heatmap_non_zero))
print(concat_strains_flat_heatmap_non_zero.shape)

(1268, 272)


In [13]:
covar = np.cov(concat_strains_flat_heatmap_non_zero, rowvar=False)
eigvals, eigvecs = np.linalg.eig(covar)
selected_eigvecs = np.fliplr(eigvecs[:, 0:3])
data_proj = concat_strains_flat_heatmap_non_zero @ selected_eigvecs
data_proj[:,1] = -data_proj[:,1]

with h5py.File(save_dir / 'pca_data_fork_plots.h5', 'w') as hdf:
    hdf.create_dataset('ref_selected_eigvecs', data = selected_eigvecs)
    

In [29]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

start_idx = 0
for i in range(0,nr_strains):
    nr_points = int(nr_non_zero_els[i])
    inds = np.arange(start_idx, nr_points+start_idx)
    scat_plt = ax.scatter(data_proj[inds, 0], data_proj[inds, 1], data_proj[inds, 2], marker='o')
    start_idx = nr_points + start_idx
 
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.legend(titles, loc='upper left', bbox_to_anchor=(0.9, 1.1))

ref_range = np.arange(0,int(nr_non_zero_els[0]))
ref_points = data_proj[ref_range, :]

ref_chull = ConvexHull(ref_points)
ref_hull_points = ref_points[ref_chull.vertices]
ref_hull_delaunay = Delaunay(ref_hull_points)
for simplex in ref_chull.simplices:
    faces = ref_points[simplex]
    poly = Poly3DCollection([faces], alpha=0.3, edgecolor='black')
    ax.add_collection3d(poly)

In [28]:
start_ind = int(nr_non_zero_els[0])
for i in range(1, nr_strains):
    nr_strain_points = int(nr_non_zero_els[i])
    inds = np.arange(start_ind, nr_strain_points+start_ind)
    strain_points = data_proj[inds, :]
    is_inside = ref_hull_delaunay.find_simplex(strain_points) >= 0
    start_ind = nr_strain_points+start_ind
    percent_points_inside = np.sum(is_inside)/len(is_inside)
    print(f"For {titles[i]} {round(percent_points_inside*100,2)}% of points are within the reference convex hull")

For ΔDARS1 1.63% of points are within the reference convex hull
For ΔDARS2 9.39% of points are within the reference convex hull
For ΔdatA 3.44% of points are within the reference convex hull


In [14]:
print(heatmap_init_ref.shape)

(8, 34)
